### 1.라이브러리 불러오기

일반 CPU 활용 시

In [1]:
import os
import cv2

In [2]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers

In [3]:
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Dropout, Flatten, Dense

In [4]:
import numpy as np
from numpy import array

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
import matplotlib.pyplot as plt

In [7]:
%matplotlib inline

### 1. 데이터 불러오기

In [8]:
(x_train, y_train), (x_test,y_test) =\
    keras.datasets.fashion_mnist.load_data()

In [9]:
## 정답지를 해석할 수 있는 리스트
## 예시 fashion_mnist_labels[9] = Ankleboot
fashion_mnist_labels = ["T-shrt/top",
                        "Trouser",
                        "Pullover",
                        "Dress",
                        "Coat",
                        "Sandal",
                        "Shirt",
                        "Sneaker",
                        "Bag",
                        "Ankle boot"]

In [10]:
fashion_mnist_labels[ y_train[0] ]

'Ankle boot'

### 2. 데이터 분리 및 정제

In [11]:
x_trainClean = x_train.astype("float32") / 255
x_testClean = x_test.astype("float32") / 255

totalTrainImgLen = x_trainClean.shape[0]
totalTestImgLen = x_testClean.shape[0]

imgSize = x_train[0].shape[0]

w,h = imgSize, imgSize

x_trainClean = x_trainClean.reshape( totalTrainImgLen, w, h, 1)
x_testClean = x_testClean.reshape( totalTestImgLen, w, h, 1)

y_trainOne = to_categorical(y_train, 10)
y_testOne = to_categorical(y_test, 10)

In [43]:
modelInputDim = x_trainClean[0].shape

In [46]:
### train 
### features -> x_trainClean
### label -> y_trainOne

In [86]:
model = keras.Sequential()

In [87]:
### 0. inputLayer! (입력데이터 첫번째 shape 들어가야한다)

In [88]:
model.add(Conv2D( filters=32, 
                  kernel_size=3,
                  padding="same",
                  activation="relu",
                  input_shape=modelInputDim))

In [89]:
### 1. Hidden Layer!

In [90]:
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.7))
model.add(Conv2D( filters=32, 
                  kernel_size=3, 
                  padding="same",
                  activation="relu"))

In [91]:
model.add(MaxPooling2D(pool_size=2))

In [92]:
model.add(Flatten())

In [93]:
### 2. Output Layer!

In [94]:
model.add(Dense(10, activation="softmax"))

In [95]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 32)        0         
                                                                 
 conv2d_12 (Conv2D)          (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1568)             

### 4. 모델 컴파일

In [97]:
# from tensorflow.keras.losses import CategoricalCrossentropy

In [98]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"] )

### 5. 모델 훈련

In [99]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [101]:
callbacks = [
    EarlyStopping( monitor="val_loss", patience=10),
    ModelCheckpoint(filepath="model_weight.h5",
                    monitor="val_accuracy", verbose=1,
                    save_best_only=True)
]

In [102]:
history = model.fit(x_trainClean,
          y_trainOne,
          epochs=10,
          batch_size=32,
          shuffle=True,
          callbacks=callbacks,
          validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - ETA: 0s - loss: 0.6076 - accuracy: 0.7788
Epoch 1: val_accuracy improved from -inf to 0.84517, saving model to model_weight.h5
1500/1500 [==============================] - 45s 29ms/step - loss: 0.6076 - accuracy: 0.7788 - val_loss: 0.4461 - val_accuracy: 0.8452
Epoch 2/10
1500/1500 [==============================] - ETA: 0s - loss: 0.4410 - accuracy: 0.8420
Epoch 2: val_accuracy improved from 0.84517 to 0.87217, saving model to model_weight.h5
1500/1500 [==============================] - 45s 30ms/step - loss: 0.4410 - accuracy: 0.8420 - val_loss: 0.3712 - val_accuracy: 0.8722
Epoch 3/10
1500/1500 [==============================] - ETA: 0s - loss: 0.3948 - accuracy: 0.8606
Epoch 3: val_accuracy improved from 0.87217 to 0.87983, saving model to model_weight.h5
1500/1500 [==============================] - 41s 27ms/step - loss: 0.3948 - accuracy: 0.8606 - val_loss: 0.3453 - val_accuracy: 0.8798
Epoch 4/10
1499/1500 [==================

### 6. 모델 추론

In [104]:
model.evaluate(x_testClean, y_testOne, verbose=1)

313/313 [==============================] - 3s 10ms/step - loss: 0.2846 - accuracy: 0.8980


[0.284613698720932, 0.8980000019073486]

### 실제 이미지 모델 예측

In [107]:
testImg = x_train[0]

In [109]:
cv2.imwrite("../images/boots.jpg", testImg)

True

In [118]:
### 0. 이미지 로드
testImgPath = "../images/boots.jpg"
testImgReal = cv2.imread(testImgPath, cv2.IMREAD_GRAYSCALE)
print(testImgReal.shape)

(28, 28)


In [119]:
testImgRealClean = testImgReal.astype("float32") / 255

In [120]:
testImgCleanComp = testImgRealClean.reshape(1,28,28,1)

In [123]:
predictValue = model.predict(testImgCleanComp)

1/1 [==============================] - 0s 34ms/step


In [126]:
predictValue

array([[1.2269193e-07, 1.1539318e-08, 1.0892919e-07, 4.3500865e-08,
        2.3492049e-07, 1.5195616e-04, 1.9122895e-06, 1.8537711e-02,
        1.3222169e-05, 9.8129463e-01]], dtype=float32)

In [128]:
predictValueMaxIndex = np.argmax( predictValue )

In [131]:
answerLabel = fashion_mnist_labels[predictValueMaxIndex]

In [132]:
answerLabel

'Ankle boot'